In [42]:

# Importieren der notwendigen Bibliotheken
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


In [43]:
# Dataset-Klasse für Gehirntumor-Bilder
class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Initialisiert den Dataset für Gehirntumor-Klassifikation.
        Args:
        - image_dir: Pfad zum Ordner 'data', der die Unterordner 'yes/' und 'no/' enthält.
        - transform: Transformationen für die Vorverarbeitung.
        """
        self.image_paths = []
        self.labels = []
        self.transform = transform

        # Pfade zu den Unterordnern "yes" (Tumor) und "no" (Healthy)
        tumor_dir = Path('data/yes/*.jpg') / "yes"
        healthy_dir = Path('data/no/*.jpg') / "no"

        # Bilder aus "yes" (Tumor) mit Label 1
        self.image_paths.extend(tumor_dir.rglob("*.jpg"))
        self.labels.extend([1] * len(list(tumor_dir.rglob("*.jpg"))))

        # Bilder aus "no" (Healthy) mit Label 0
        self.image_paths.extend(healthy_dir.rglob("*.jpg"))
        self.labels.extend([0] * len(list(healthy_dir.rglob("*.jpg"))))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformationen für die Datenvorbereitung
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [44]:

# Transfer Learning Modell mit ResNet-18
class TransferLearningModel(nn.Module):
    def __init__(self):
        super(TransferLearningModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.base_model(x)

# Initialisierung von Modell, Loss-Funktion und Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransferLearningModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.base_model.fc.parameters(), lr=0.001)


In [45]:

# Trainings- und Validierungsprozess
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    """
    Trainiert das Modell und validiert es in jedem Epoch.
    Args:
    - model: Das zu trainierende Modell.
    - dataloaders: Dictionary mit 'train' und 'val' Dataladern.
    - criterion: Loss-Funktion.
    - optimizer: Optimizer für das Training.
    - num_epochs: Anzahl der Epochen.
    """
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model


In [46]:
# Dataset-Klasse für Gehirntumor-Bilder
class BrainTumorDataset(Dataset):
    def __init__(self, transform=None):
        """
        Initialisiert den Dataset für Gehirntumor-Klassifikation.
        Args:
        - transform: Transformationen für die Vorverarbeitung.
        """
        self.image_paths = []
        self.labels = []
        self.transform = transform

        # Direktes Laden der Bildpfade
        tumor_dir = Path('data/yes')  # Pfad zu Tumor-Bildern
        healthy_dir = Path('data/no')  # Pfad zu gesunden Bildern

        # Bilder aus "yes" (Tumor) mit Label 1
        self.image_paths.extend(list(tumor_dir.rglob("*.jpg")))
        self.labels.extend([1] * len(list(tumor_dir.rglob("*.jpg"))))

        # Bilder aus "no" (Healthy) mit Label 0
        self.image_paths.extend(list(healthy_dir.rglob("*.jpg")))
        self.labels.extend([0] * len(list(healthy_dir.rglob("*.jpg"))))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Transformationen für die Datenvorbereitung
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Laden des Datasets
dataset = BrainTumorDataset(transform=data_transforms)

# Debugging: Gesamtanzahl der Bilder überprüfen
print(f"Gesamtanzahl der Bilder im Dataset: {len(dataset)}")
print(f"Anzahl der Tumor-Bilder: {sum(label == 1 for label in dataset.labels)}")
print(f"Anzahl der gesunden Bilder: {sum(label == 0 for label in dataset.labels)}")

# Prüfen, ob genügend Bilder für das Training und die Validierung vorhanden sind
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

if train_size == 0 or val_size == 0:
    raise ValueError("Trainings- oder Validierungsset hat keine ausreichenden Samples!")

# Aufteilen des Datasets in Trainings- und Validierungsdaten
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader für Training und Validierung erstellen
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=16, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=16, shuffle=False)
}

# Debugging: Überprüfen der DataLoader-Größen
print(f"Anzahl der Trainings-Samples: {len(train_dataset)}")
print(f"Anzahl der Validierungs-Samples: {len(val_dataset)}")

# Training starten
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)


Gesamtanzahl der Bilder im Dataset: 800
Anzahl der Tumor-Bilder: 392
Anzahl der gesunden Bilder: 408
Anzahl der Trainings-Samples: 640
Anzahl der Validierungs-Samples: 160
Epoch 1/40
----------
train Loss: 0.4796 Acc: 0.7828
val Loss: 0.2580 Acc: 0.9125
Epoch 2/40
----------
train Loss: 0.3337 Acc: 0.8562
val Loss: 0.1754 Acc: 0.9375
Epoch 3/40
----------
train Loss: 0.3033 Acc: 0.8672
val Loss: 0.1972 Acc: 0.9250
Epoch 4/40
----------
train Loss: 0.2822 Acc: 0.8859
val Loss: 0.1775 Acc: 0.9500
Epoch 5/40
----------
train Loss: 0.2306 Acc: 0.9031
val Loss: 0.1114 Acc: 0.9625
Epoch 6/40
----------
train Loss: 0.2423 Acc: 0.8938
val Loss: 0.1300 Acc: 0.9563
Epoch 7/40
----------
train Loss: 0.2619 Acc: 0.8969
val Loss: 0.1553 Acc: 0.9250
Epoch 8/40
----------
train Loss: 0.2461 Acc: 0.9062
val Loss: 0.1494 Acc: 0.9375
Epoch 9/40
----------
train Loss: 0.2062 Acc: 0.9250
val Loss: 0.0927 Acc: 0.9750
Epoch 10/40
----------
train Loss: 0.2079 Acc: 0.9203
val Loss: 0.1429 Acc: 0.9313
Epoch 1

In [47]:

# Speichern des Modells
torch.save(model.state_dict(), "brain_tumor_model.pth")
print("Modell gespeichert.")


Modell gespeichert.


In [49]:
from PIL import Image
import torch
from torchvision import transforms
from pathlib import Path

# Transformationen wie während des Trainings
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Modell laden
model = TransferLearningModel()  # Erstellen Sie die Modellinstanz
model.load_state_dict(torch.load("brain_tumor_model.pth"))  # Pfad zum gespeicherten Modell
model.eval()  # Schalten Sie das Modell in den Evaluierungsmodus

# Gerät festlegen (GPU oder CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Pfad zum Testbild
test_image_path = Path("data/new/image.jpg")  # Pfad zum Testbild

# Bild laden und vorbereiten
image = Image.open(test_image_path).convert("RGB")  # Bild öffnen und in RGB konvertieren
input_tensor = data_transforms(image).unsqueeze(0).to(device)  # Transformation und Batch-Dimension hinzufügen

# Vorhersage
with torch.no_grad():
    output = model(input_tensor)  # Modellvorhersage
    _, predicted_class = torch.max(output, 1)  # Klasse mit der höchsten Wahrscheinlichkeit

# Ausgabe der Vorhersage
if predicted_class.item() == 1:
    print("Tumor")
else:
    print("Healthy")


Healthy


C:\Users\vulpe\AppData\Local\Temp\ipykernel_34676\4132344695.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("brain_tumor_model.pth")) 